In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import os
from tqdm import tqdm

In [3]:
data = pd.read_csv('processed_data.csv')

In [9]:
X = data['join_processed_review'].astype(str).tolist()
y = data['overall'].astype(int)

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [49]:
lengths = [len(seq) for seq in sequences]
maxlen = int(np.percentile(lengths, 95))
X = pad_sequences(sequences, maxlen=maxlen)
y = to_categorical(y)

In [51]:
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  
embedding_dim = 300

In [53]:
embedding_index = {}
with open('wiki-news-300d-1M-subword.vec', encoding='utf-8') as f:
    next(f)  # Skip header
    for line in tqdm(f):
        values = line.rstrip().split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

999994it [01:13, 13645.70it/s]


In [54]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in embedding_index:
        embedding_matrix[i] = embedding_index[word]

In [55]:
model = Sequential()
model.add(Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=300,
    trainable=False
))

C:\Users\ling1\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [56]:
model = Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=300, 
                    weights=[embedding_matrix], input_length=maxlen, trainable=False))

model.add(LSTM(128, return_sequences=True, dropout=0.4, recurrent_dropout=0.4))

model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(32, activation='relu'))

model.add(Dense(6, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2, verbose=2)

Epoch 1/10
123/123 - 19s - 157ms/step - accuracy: 0.7936 - loss: 0.7637 - val_accuracy: 0.7884 - val_loss: 0.7937
Epoch 2/10
123/123 - 16s - 129ms/step - accuracy: 0.8005 - loss: 0.7418 - val_accuracy: 0.7884 - val_loss: 0.7838
Epoch 3/10
123/123 - 13s - 105ms/step - accuracy: 0.8005 - loss: 0.7348 - val_accuracy: 0.7884 - val_loss: 0.7804
Epoch 4/10
123/123 - 11s - 92ms/step - accuracy: 0.8005 - loss: 0.7337 - val_accuracy: 0.7884 - val_loss: 0.7918
Epoch 5/10
123/123 - 11s - 86ms/step - accuracy: 0.8005 - loss: 0.7357 - val_accuracy: 0.7884 - val_loss: 0.7836
Epoch 6/10
123/123 - 10s - 78ms/step - accuracy: 0.8005 - loss: 0.7320 - val_accuracy: 0.7884 - val_loss: 0.7764
Epoch 7/10
123/123 - 9s - 76ms/step - accuracy: 0.8005 - loss: 0.7311 - val_accuracy: 0.7884 - val_loss: 0.7781
Epoch 8/10
123/123 - 10s - 79ms/step - accuracy: 0.8005 - loss: 0.7310 - val_accuracy: 0.7884 - val_loss: 0.7783
Epoch 9/10
123/123 - 9s - 72ms/step - accuracy: 0.8005 - loss: 0.7203 - val_accuracy: 0.7884 -

In [57]:
val_accuracy = history.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {val_accuracy * 100:.2f}%")

Final Validation Accuracy: 78.84%


In [58]:
train_accuracy = history.history['accuracy'][-1]
print(f"Final Training Accuracy: {train_accuracy * 100:.2f}%")

Final Training Accuracy: 80.05%
